In [1]:
import pandas as pd
import os
from nltk import sent_tokenize
import re

In [2]:
df = pd.DataFrame(columns=['predict', 'ground_truth'])

In [3]:
path = r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\data\output_fastabs\output'
files = os.listdir(path)
files.sort(key=lambda f: int(re.sub('\D', '', f)))
predict = []
for f in files:
    with open(os.path.join(path, f), 'r', encoding='utf8') as sum:
        data = sum.read()
    predict.append(data)

In [4]:
df['predict'] = predict

In [6]:
label = pd.read_csv(r'C:\Users\anhdq33\Downloads\VinBigData\Deep_Learning\Project\data\raw_data\data\test.csv', index_col=0)
label.head()

,summary,fullText
0,Bản_án cho đối_tượng giả_danh công_an để lừa_đảo,"Ngày 25/2 , TAND TP. Đà_Nẵng tuyên_phạt Hồ_Xuâ..."
1,Nam thanh_niên thủ_dâm trên xe_buýt từng bị xử...,Phát_hiện nam thanh_niên đang thủ_dâm trên xe_...
2,"Không được công_nhận , thuốc "" sinh con theo ý...",“ Baby_Support ” và “ Hello baby ” là hai cái ...
3,VEC từ_chối phục_vụ vĩnh_viễn 2 ô_tô gây_rối t...,Liên_quan đến vụ gây_rối tại trạm thu phí trên...
4,VKSND tỉnh Sóc_Trăng xin_lỗi oan sai “ nữ_hoàn...,Bà Huỳnh_Ngọc_Bích được mọi người biết đến là ...


In [9]:
df['ground_truth'] = label['summary']

In [10]:
df.tail()

,predict,ground_truth
22639,mưa_đá ở nghệ_an,Mưa_đá trút xuống vùng_biên Nghệ_An làm hỏng n...
22640,tp hcm cấm tàu chạy trên sông sài_gòn,Tàu_cánh_ngầm TP HCM - Vũng_Tàu được chạy trở_lại
22641,chờ cơ_chế miễn học_phí cấp thcs,TP HCM nghiên_cứu chính_sách miễn học_phí THPT
22642,kiều_bào : ' kiều_bào là khơi gợi nhiệt_huyết ...,Thứ_trưởng Trần_Văn_Tùng : ' Tạo mọi thuận_lợi...
22643,thanh_tra xây_dựng cầu_vượt ở hà_nội : không l...,Hà_Nội phủ_nhận sai_phạm trong xây_dựng cầu_vượt


In [11]:
!pip install rouge-score

In [12]:
import numpy as np
from rouge_score import rouge_scorer
from datetime import datetime

def rouge_scoring(df, raw_col, machine_col):
    scr_type = ['rouge1', 'rouge2', 'rougeL']
    scorer = rouge_scorer.RougeScorer(scr_type, use_stemmer=True)

    rouge_score = {'rouge1':{'precision':[],
                             'recall':[],
                             'fmeasure': []},
                   'rouge2':{'precision':[],
                             'recall':[],
                             'fmeasure': []},
                   'rougeL':{'precision':[],
                             'recall':[],
                             'fmeasure': []}}
    
    for i, r in df.iterrows():
        score_all = scorer.score(r[raw_col],r[machine_col])
        for t in scr_type:
                rouge_score[t]['precision'].append(score_all[t].precision)
                rouge_score[t]['recall'].append(score_all[t].recall)
                rouge_score[t]['fmeasure'].append(score_all[t].fmeasure)
    for t in scr_type:
        for k, v in rouge_score[t].items():
            rouge_score[t][k] = np.mean(np.array(v))
            print('{} ==== {}-{}: {}'.format(datetime.now(),t.upper(),k.upper(), np.mean(np.array(v))))
    return rouge_score

In [15]:
df = df.dropna()

In [16]:
rouge_score = rouge_scoring(df, 'ground_truth', 'predict')

2021-01-15 22:36:34.221659 ==== ROUGE1-PRECISION: 0.5676503097707387
2021-01-15 22:36:34.226664 ==== ROUGE1-RECALL: 0.5247372345428319
2021-01-15 22:36:34.231618 ==== ROUGE1-FMEASURE: 0.5335343900763853
2021-01-15 22:36:34.237869 ==== ROUGE2-PRECISION: 0.3152606714084578
2021-01-15 22:36:34.242860 ==== ROUGE2-RECALL: 0.2898246777769599
2021-01-15 22:36:34.248083 ==== ROUGE2-FMEASURE: 0.2950254740839309
2021-01-15 22:36:34.254121 ==== ROUGEL-PRECISION: 0.45303562570627065
2021-01-15 22:36:34.261083 ==== ROUGEL-RECALL: 0.4186615735482314
2021-01-15 22:36:34.269078 ==== ROUGEL-FMEASURE: 0.4256718528908243
